In [1]:
!pip install flashtext
!pip install scikit-learn
!pip install matplotlib

You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [8]:
import re
import pandas as pd
from helpers import yake_helper
from datetime import datetime, timedelta
from math import sqrt, floor
from sklearn.cluster import SpectralClustering
import numpy as np
import itertools
from matplotlib import pyplot as plt
from remove_polite_posts import remove_polite_posts as rpp
from flashtext.keyword import KeywordProcessor
import string
import nltk
import math
import warnings

warnings.filterwarnings('ignore')

In [5]:
forum_posts = pd.read_csv('ForumMessages.csv')

w2v_vectors = pd.read_csv('fine-tuning-word2vec/forum_posts_word2vec.model',
                          delim_whitespace=True,
                          skiprows=[0],
                          header=None)

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3173: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
w2v_vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,p,-0.258226,1.322337,0.291533,-0.679372,0.362768,0.961267,0.694033,-0.019635,-0.007253,...,0.263582,-0.606414,0.356946,-0.168584,-0.717055,-1.616924,-0.834215,-0.698412,-0.177316,-0.203391
1,the,1.273628,-0.651237,-0.065435,-0.083179,1.152159,-0.257050,1.377131,1.871087,0.498178,...,0.166710,-1.062766,1.464719,-0.346945,-0.520129,-1.788661,-0.298106,-0.374428,0.806349,-2.619995
2,a,0.643546,-1.170918,0.753471,0.290148,0.276544,1.110373,3.050881,1.799117,-0.873134,...,0.684108,-0.937324,0.919357,0.979155,-1.046691,-1.881935,0.662633,0.407499,1.435630,-1.053839
3,i,1.832707,-0.055822,0.288658,0.215443,2.397752,2.606470,-1.467209,1.140652,-0.241518,...,-0.217198,1.053774,1.247309,0.098720,0.297497,-2.680148,-1.205078,0.714816,0.450466,-1.331006
4,to,2.431875,0.986299,-1.254033,0.172852,0.473982,-0.299878,1.807054,0.864406,-1.208238,...,0.171002,0.671840,0.925534,-1.304466,0.659252,-0.420799,0.295289,0.453520,0.407416,-2.202719


In [11]:
w2v_vectors.index = w2v_vectors[0]
w2v_vectors.drop(0, axis=1, inplace=True)

In [ ]:
def vectors_from_post(post):
    all_words = []
    for word in post:
        all_words.append(word)
    
    return(w2v_vectors[w2v_vectors.index.isin(all_words)])

def get_keyword_set_by_cluster(number):
    cluster_index = list(clustering.labels_ == number)
    return(list(itertools.compress(keyword_sets, cluster_index)))

def get_post_info_by_cluster(number, data, cluster):
    return(data[cluster.labels_ == number])

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return(re.sub(clean, '', text))

def remove_thanks(text):
    text = text.lower()
    text = re.sub("nice", "", text)
    text = re.sub("thank.*\s", " ", text)
    text = re.sub("good","", text)
    text = re.sub("hi", "", text)
    text = re.sub("hello", "", text)
    return (text)

def polite_post_index(forum_posts):
    polite_indexes = []
    
    stop_word_list = ["no problem", "thanks", "thx", "thank", "great",
                      "nice", "interesting", "awesome", "perfect", 
                      "amazing", "well done", "good job"]

    keyword_processor = KeywordProcessor()
    keyword_processor.add_keywords_from_list(stop_word_list)

    for i,post in enumerate(forum_posts):
        post = post.lower().translate(str.maketrans({a:None for a in string.punctuation}))        
        if len(post) < 100:
            keywords_found = keyword_processor.extract_keywords(post.lower(), span_info=True)
            if keywords_found:
                polite_indexes.append(i)

    return (polite_indexes)